>- It's not a good practice to login for scraping unless its absolutely necessary

In [24]:
## load libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
import time
import os
import pandas as pd
import numpy as np


In [10]:
## login to twitter

web = "https://twitter.com/i/flow/login"
path = 'C:/Users/cmwak/chrome_driver/chromedriver'

driver = webdriver.Chrome(service = Service(path))

driver.maximize_window()

driver.get(web)


# wait of 6 seconds to let the page load the content
time.sleep(6)  # this time might vary depending on your computer

# locating username and password inputs and sending text to the inputs
# username
username = driver.find_element(By.XPATH, '//input[@autocomplete ="username"]')
username.send_keys("your_user_name")  # Write Email or user name Here
# username.send_keys(os.environ.get("TWITTER_USER"))

# Clicking on "Next" button
next_button = driver.find_element(By.XPATH, '//div[@role="button"]//span[text()="Next"]')
next_button.click()

# wait of 2 seconds after clicking button
time.sleep(2)

# password
password = driver.find_element(By.XPATH, '//input[@autocomplete ="current-password"]')
password.send_keys("your_password_here")  # Write Password Here
# password.send_keys(os.environ.get("TWITTER_PASS"))

# locating login button and then clicking on it
login_button = driver.find_element(By.XPATH, ('//div[@role="button"]//span[text()="Log in"]'))
login_button.click()

time.sleep(10)

## searching for a keyword
search_input = driver.find_element(By.XPATH, '//input[@aria-label="Search query"]')
search_input.send_keys("IkoKaziKe")

time.sleep(6)

search_input.send_keys(Keys.RETURN)

time.sleep(6)

## now trigger the scroll
# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    time.sleep(6)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("Reached bottom of page")
        break
    else:
        print("Scrolling")
        
    time.sleep(6)
    last_height = new_height




# closing driver
driver.quit()


Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Scrolling
Reached bottom of page


## Elements that are visible

In [13]:
## we are going to scrape based on the query we have searched for

### get the latest instead of the top tweets
web = "https://twitter.com/search?q=%23IkoKaziKE&src=typed_query&f=top"
path = 'C:/Users/cmwak/chrome_driver/chromedriver'

driver = webdriver.Chrome(service = Service(path))
driver.maximize_window()

driver.get(web)

## time.sleep(6)
## close_popup = driver.find_element_by_xpath('//div[@aria-label="Close"]')
## time.sleep(6)
## close_popup.click()


## time.sleep(3)

# wait for the notification popup to appear and then close it
try:
    not_now_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Close"]'))
    )
    not_now_button.click()
except:
    print("Notification popup not found or couldn't be closed.")

## sleep for 3 seconds to let the page load the content
time.sleep(6)

tweets = driver.find_elements(By.XPATH, '//div[contains(@class, "r-kzbkwu")]')


## create the list to store the data

user_name = []
tweet_text = []
date_posted = []

try:
    for tweet in tweets:
            ## user name
        username = tweet.find_element(By.XPATH, './/span[contains(text(), "@")]').text
        user_name.append(username)
        ## text inside a tweet
        text = tweet.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text
        tweet_text.append(text)
        ## date of the post

        try:
            date = tweet.find_element(By.XPATH, './/time[contains(@datetime, "2023")]').text
            date_posted.append(date)
        except NoSuchElementException:
            date_posted.append(np.nan)
except NoSuchElementException:
    print("No tweets found")
        

# closing driver
driver.quit()

In [14]:
print(user_name)
print(tweet_text)
print(date_posted)

['@FabrizioRomano', '@elonmusk', '@DailyLoud', '@PopCrave']
['Jude Bellingham: “I was at the final when Madrid beat Liverpool, that was a huge factor in my decision to join Real”.  #RealMadrid\n\n“I’m joining the greatest club in the world and it’s not about money”.', '“What are you thinking?”', 'BREAKING: The Rubik’s cube world record has been broken at 3.13 seconds ', 'Tom Holland on his ‘Lip\u202fSync Battle’ performance from 2017:\n\n“I’m proud of it ... I don’t give a fuck. I’ve grown up in the most non-toxic-masculine environment possible. I didn’t realize what I was doing was so forward-thinking. I was just like, ‘Yeah, fuck it, I’ll put some…']
['3h', '14h', '16h', '10h']


## Get the tweets using a function instead

In [15]:
## we are going to scrape based on the query we have searched for

### get the latest instead of the top tweets
web = "https://twitter.com/search?q=%23IkoKaziKE&src=typed_query&f=top"
path = 'C:/Users/cmwak/chrome_driver/chromedriver'

driver = webdriver.Chrome(service = Service(path))
driver.maximize_window()

driver.get(web)

## time.sleep(6)
## close_popup = driver.find_element_by_xpath('//div[@aria-label="Close"]')
## time.sleep(6)
## close_popup.click()


## time.sleep(3)

# wait for the notification popup to appear and then close it
try:
    not_now_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Close"]'))
    )
    not_now_button.click()
except:
    print("Notification popup not found or couldn't be closed.")

## sleep for 3 seconds to let the page load the content
time.sleep(10)

tweets = driver.find_elements(By.XPATH, '//div[contains(@class, "r-kzbkwu")]')


## create the list to store the data

def get_tweets(element):
    try:

        """This function takes in a tweet element and returns the username, text, and date posted"""

        username = element.find_element(By.XPATH, './/span[contains(text(), "@")]').text
        text = element.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text
        date = element.find_element(By.XPATH, './/time[contains(@datetime, "2023")]').text
        
        tweet_data = [username, text, date]



    except (NoSuchElementException, StaleElementReferenceException):
        tweet_data = [np.nan, np.nan, np.nan]
    
    return tweet_data

user_name_data = []
tweet_text_data = []
date_posted_data = []

for tweet in tweets:
    tweet_data = get_tweets(tweet)

    user_name_data.append(tweet_data[0])
    ## append to tweet_text_data and remove the \n
    tweet_text_data.append(tweet_data[1])
    date_posted_data.append(tweet_data[2])



driver.quit()

In [19]:
## create a dataframe
df_tweets = pd.DataFrame({'user_name': user_name, 'tweet_text': tweet_text, 'date_posted': date_posted})
df_tweets.to_csv('tweets.csv', index=False)
df_tweets

,user_name,tweet_text,date_posted
0,@FabrizioRomano,Jude Bellingham: “I was at the final when Madr...,3h
1,@elonmusk,“What are you thinking?”,14h
2,@DailyLoud,BREAKING: The Rubik’s cube world record has be...,16h
3,@PopCrave,Tom Holland on his ‘Lip Sync Battle’ performan...,10h


## Infinite Scrolliog

In [22]:
## trigger infinite scroll

## we are going to scrape based on the query we have searched for

### get the latest instead of the top tweets
web = "https://twitter.com/KenyaPower_Care"
path = 'C:/Users/cmwak/chrome_driver/chromedriver'

driver = webdriver.Chrome(service = Service(path))

driver.maximize_window()

driver.get(web)

## time.sleep(6)
## close_popup = driver.find_element_by_xpath('//div[@aria-label="Close"]')
## time.sleep(6)
## close_popup.click()


## time.sleep(3)

# wait for the notification popup to appear and then close it
try:
    not_now_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Close"]'))
    )
    not_now_button.click()
except:
    print("Notification popup not found or couldn't be closed.")


## scroll down to the bottom of the page
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    try:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(6)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(6)

        if new_height == last_height:
            print("Reached the end of the page")
            break
        else:
            print("Scrolling down...")
            
        last_height = new_height
    except NoSuchWindowException:
        print("Window closed")
        break



tweets = driver.find_elements(By.XPATH, '//div[contains(@class, "r-kzbkwu")]')


driver.quit()

Scrolling down...
Scrolling down...
Window closed


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D0A813+48355]
	(No symbol) [0x00C9C4B1]
	(No symbol) [0x00BA5358]
	(No symbol) [0x00B8D293]
	(No symbol) [0x00BEE37B]
	(No symbol) [0x00BFC473]
	(No symbol) [0x00BEA536]
	(No symbol) [0x00BC82DC]
	(No symbol) [0x00BC93DD]
	GetHandleVerifier [0x00F6AABD+2539405]
	GetHandleVerifier [0x00FAA78F+2800735]
	GetHandleVerifier [0x00FA456C+2775612]
	GetHandleVerifier [0x00D951E0+616112]
	(No symbol) [0x00CA5F8C]
	(No symbol) [0x00CA2328]
	(No symbol) [0x00CA240B]
	(No symbol) [0x00C94FF7]
	BaseThreadInitThunk [0x765B7D59+25]
	RtlInitializeExceptionChain [0x77B8B74B+107]
	RtlClearBits [0x77B8B6CF+191]


>- The above does infinite scrolling
>- My next focus is the infinite scrolling that is triggered by another button like load_more button or search

Something else, to ensure the infinite scrolling is continues, make sure you have waits either implicit or explicit to ensure the contents being scrolled is loaded

In [74]:
## change website to this


## trigger infinite scroll

## we are going to scrape based on the query we have searched for

### get the latest instead of the top tweets
web = "https://twitter.com/search?q=ikokazike&src=typeahead_click"
path = 'C:/Users/cmwak/chrome_driver/chromedriver'

driver = webdriver.Chrome(service = Service(path))

driver.get(web)
driver.maximize_window()

## time.sleep(6)
## close_popup = driver.find_element_by_xpath('//div[@aria-label="Close"]')
## time.sleep(6)
## close_popup.click()


## time.sleep(3)

# wait for the notification popup to appear and then close it
try:
    not_now_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Close"]'))
    )
    not_now_button.click()
except:
    print("Notification popup not found or couldn't be closed.")


## scroll down to the bottom of the page
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(3)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")

    time.sleep(6)

    if new_height == last_height:
        break
    last_height = new_height




tweets = driver.find_elements(By.XPATH, '//div[contains(@class, "r-kzbkwu")]')

driver.quit()

C:\Users\cmwak\AppData\Local\Temp\ipykernel_2544\1069753328.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
C:\Users\cmwak\AppData\Local\Temp\ipykernel_2544\1069753328.py:57: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  tweets = driver.find_elements_by_xpath('//div[contains(@class, "r-kzbkwu")]')


>- For most websites using the infinite scrolling, they mostly load the contents of the page we are currently in, like twitter
>- This means that we will get the data for the page we are in (in this case tweets)
>- We will have to scrape each page as we move down the page and not wait till the end of the page to scrape everything
>- Unfortunately, one of the disadvantages of this is that we might get duplicate data because selenium might catch elements it already located before from the previous page
>- To avoid this, we will have to check if the data we are getting is already in the list of data we have gotten before
>- We will handle these duplicates with sets





In [25]:
## change website to this


## trigger infinite scroll

## we are going to scrape based on the query we have searched for

### get the latest instead of the top tweets
web = "https://twitter.com/search?q=ikokazike&src=typeahead_click"
path = 'C:/Users/cmwak/chrome_driver/chromedriver'

driver = webdriver.Chrome(service = Service(path))
driver.maximize_window()
driver.get(web)


## time.sleep(6)
## close_popup = driver.find_element_by_xpath('//div[@aria-label="Close"]')
## time.sleep(6)
## close_popup.click()


## time.sleep(3)

# wait for the notification popup to appear and then close it
try:
    not_now_button = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@aria-label="Close"]'))
    )
    not_now_button.click()
except:
    print("Notification popup not found or couldn't be closed.")


tweets = driver.find_elements(By.XPATH, '//div[contains(@class, "r-kzbkwu")]')


## create the list to store the data

def get_tweets(element):
    try:

        """This function takes in a tweet element and returns the username, text, and date posted"""

        time.sleep(3)
        username = element.find_element(By.XPATH, './/span[contains(text(), "@")]').text
        text = element.find_element(By.XPATH, './/div[@data-testid="tweetText"]').text
        date = element.find_element(By.XPATH, './/time[contains(@datetime, "2023")]').text
        
        tweet_data = [username, text, date]



    except NoSuchElementException:
        tweet_data = [np.nan, np.nan, np.nan]
    
    return tweet_data

user_name_data = []
tweet_text_data = []
date_posted_data = []

tweet_ids = set()

scrolling = True

while scrolling:

    try:

        for tweet in tweets:
        
            tweet_data = get_tweets(tweet)

            tweet_id = tweet_data[0]

            if tweet_id not in tweet_ids:

                tweet_ids.add(tweet_id)
                user_name_data.append(tweet_data[0])
                ## append to tweet_text_data and remove the \n
                tweet_text_data.append(tweet_data[1])
                date_posted_data.append(tweet_data[2])
        
    except (StaleElementReferenceException, NoSuchWindowException, ElementClickInterceptedException):
        pass   
    ## scroll down to the bottom of the page
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page 
        
        time.sleep(6)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(6)
        
        if new_height == last_height:
            scrolling = False
            break
        else:
            last_height = new_height
            break
            

driver.quit()

In [26]:
## create a dataframe
df = pd.DataFrame({'user_name': user_name_data, 'tweet_text': tweet_text_data, 'date_posted': date_posted_data})
df

,user_name,tweet_text,date_posted
